In [1]:
import sqlite3
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

In [ ]:
connection = sqlite3.connect('../chembl_29_sqlite (1)/chembl_29_sqlite/chembl_29.db')
cursor = connection.cursor()
query = "SELECT ACT.ACTIVITY_ID, ASY.ASSAY_ID, ACT.MOLREGNO , ASY.CELL_ID, ACT.STANDARD_VALUE,ACT.STANDARD_UNITS"
    ", ACT.STANDARD_TYPE " \
    " FROM ACTIVITIES ACT INNER JOIN ASSAYS ASY ON ACT.ASSAY_ID = ASY.ASSAY_ID" \
    " WHERE ASY.CELL_ID IS NOT NULL"
cursor.execute(query)
activity_dictionary = cursor.fetchall()
activity_columns =['activity_id','assay_id','molregno','cell_id','std_value','std_units','std_type']
activity_df = pd.DataFrame(activity_dictionary,activity_columns)

In [ ]:
raw_act = activity_df[activity_df['std_type'].isin(['IC50','CC50','EC50','GI50'])]

In [ ]:
raw_act = raw_act[raw_act['std_units'].isin(['nM', '10^4M', '/uM', "10'-11uM", "10'10uM", "10'8pM", 
                                             "10'7pM", "10'6pM", "10'5pM", "10'-4nM", "10'6uM", "10'5uM", 'µM'])]

In [ ]:
#TRANSFORMACIONES A uM

# nM
raw_act.loc[ raw_act['std_units'] == 'nM',['std_value']] * 0.001
raw_act.loc[ raw_act['std_units'] == 'nM',['std_units']] = 'uM'

# 10^4M 
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_value']] * 10**4
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_value']] * 1000000
raw_act.loc[ raw_act['std_units'] == '10^4M',['std_units']] = 'uM'

# /uM 
raw_act.loc[ raw_act['std_units'] == '/uM',['std_units']] = 'uM'

# 10'-11uM 
raw_act.loc[ raw_act['std_units'] == "10'-11uM",['std_value']] * 10**-11
raw_act.loc[ raw_act['std_units'] == "10'-11uM",['std_units']] = 'uM'

# 10'10uM
raw_act.loc[ raw_act['std_units'] == "10'10uM",['std_value']] * 10**10
raw_act.loc[ raw_act['std_units'] == "10'10uM",['std_units']] = 'uM'

# 10'8pM  
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_value']] * 10**8
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_value']] * 1000000
raw_act.loc[ raw_act['std_units'] == "10'8pM",['std_units']] = 'uM'

# 10'7pM
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_value']] * 10**7
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'7pM",['std_units']] = 'uM'

# 10'6pM 
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_value']] * 10**6
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'6pM",['std_units']] = 'uM'

# 10'5pM    
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_value']] * 10**5
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_value']] * 0.000001
raw_act.loc[ raw_act['std_units'] == "10'5pM",['std_units']] = 'uM'

# 10'-4nM  
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_value']] * 10**-4
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_value']] * 0.001
raw_act.loc[ raw_act['std_units'] == "10'-4nM",['std_units']] = 'uM'

# 10'6uM  
raw_act.loc[ raw_act['std_units'] == "10'6uM",['std_value']] * 10**6
raw_act.loc[ raw_act['std_units'] == "10'6uM",['std_units']] = 'uM'

# 10'5uM
raw_act.loc[ raw_act['std_units'] == "10'5uM",['std_value']] * 10**5
raw_act.loc[ raw_act['std_units'] == "10'5uM",['std_units']] = 'uM'

# µM 
raw_act.loc[ raw_act['std_units'] == "µM",['std_units']] = 'uM'

In [ ]:
raw_act['actividad']= np.where(raw_act['std_value']<=10.0,1,-1)

In [ ]:
act = pd.DataFrame(raw_act[['molregno','cell_id','active']])
act.to_csv('../csvs/summary30.csv',header=True,index=True)
act=pd.read_csv('../csvs/summary30.csv',index_col=0).drop_duplicates(keep='first')
df_count = act.groupby(by=['molregno','cell_id'])
index1 = pd.MultiIndex.from_arrays([act[col] for col in ['molregno', 'cell_id']])
index2 = df_count.index
summ_act=act.loc[index1.isin(index2)]

In [ ]:
size = summ_act.shape[0]
print (len(summ_act[summ_act['active'].isin([1])])/size)
print (len(summ_act[summ_act['active'].isin([-1])])/size)

In [ ]:
comp_in_act = summary.groupby(by=['compound_id', 'active']).size().groupby(by=['compound_id'])
        .size()[summary.groupby(by=['compound_id', 'active']).size().groupby(by=['compound_id']).size()==2].index
b_summary = summary[summary['compound_id'].isin(comp_in_act)]
b_summary

In [ ]:
lista_compuestos = []
for i in b_summary['cell_id'].unique(): #i es el cell id
    lista_compuestos.append([i, list(b_summary[b_summary['cell_id' ]== i]['compound_id']), 
                             list(b_summary[b_summary['cell_id' ]== i]['active'])])

In [ ]:
index1 = pd.MultiIndex.from_arrays([v30[col] for col in ['compound_id', 'cell_id']])
index2 = pd.MultiIndex.from_arrays([v29[col] for col in ['compound_id', 'cell_id']])
v30_ni = v30.loc[~index1.isin(index2)]
v30_ni.columns =['compound_id','cell_id','active']
v30_ni

In [ ]:
################# esto es el algoritmo del sistema de recomendacion
conj_to_predict = pd.read_csv('../csvs/compounds30_ni_29.csv', index_col = 0)
top3 = pd.read_csv('../csvs/comp_sim_df_v30.csv', index_col=0)
perfil_afinidad_activity = pd.read_csv('../csvs/perfil_afinidad_celulas_29_plusActivity, Obtener perfiles de afinidad.csv')
conj_to_predict = conj_to_predict.sort_values('compound_id')
conj_to_predict = conj_to_predict.assign(prediccion='')
conj_to_predict.reset_index(drop=True, inplace=True)
#esta función retorna la lista de compuestos que interactuan con una celula, además de la actividad
def get_listaCompuestos_conActividad(cell):
    try:
        lista_compuestos_actividad = 
        [eval(perfil_afinidad_activity['id_compuestos_actividad'][perfil_afinidad_activity['cell_id']==cell].tolist()[0]),
                        eval(perfil_afinidad_activity['activity'][perfil_afinidad_activity['cell_id']==cell].tolist()[0])]
    except:
        lista_compuestos_actividad = []
    return lista_compuestos_actividad
#esta función retorna los compuestos (en forma de lista) pertenecientes al top3 del compuesto que se envia como parámetro
def get_top3_compound(compuesto):
    top3_compuesto = []
    try:
        top3_compuesto = eval(top3['top3'][top3['comp_id'] == compuesto].tolist()[0])
    except:
        pass
    return top3_compuesto

In [ ]:
for i in conj_to_predict.index: #Recorro el conjunto de datos a predecir
    array_interaccion = []
    compuesto_a_predecir = conj_to_predict['compound_id'][i] #Par compuesto - línea celular al que se 
    linea_celular_a_predecir = conj_to_predict['cell_id'][i] #le va a realizar la predicción
    perfil_afinidad = get_listaCompuestos_conActividad(linea_celular_a_predecir) #Obtención del perfil de afinidad
    if perfil_afinidad:
        for j in perfil_afinidad[0]:
            top3_cada_compuesto = get_top3_compound(j)#Obtención del top3 de cada compuesto del perfil de afinidad
            if top3_cada_compuesto:
                if compuesto_a_predecir in top3_cada_compuesto:
                    array_interaccion.append(perfil_afinidad[0].index(j))#Aquí se guarda el índice del valor de la interacción
    else:
        conj_to_predict.at[i, 'prediccion'] = 'NA' #En caso de que no exista un perfil de afinidad para la línea celular
        
    if len(array_interaccion) > 0:
        if len(array_interaccion) == 1: #Si existe solo un valor de interacción, se coloca ese valor en la predicción
            conj_to_predict.at[i, 'prediccion'] = perfil_afinidad[1][array_interaccion[0]]
        elif len(array_interaccion) > 1: #Se verifica que todos los valores de interacción sean iguales
            resultado = all(element == array_interaccion[0] for element in array_interaccion)
            if (resultado): #En caso de que todos los valores sean iguales
                conj_to_predict.at[i, 'prediccion'] = perfil_afinidad[1][array_interaccion[0]]
            else: #En caso de que no todos los valores sean iguales
                conj_to_predict.at[i, 'prediccion'] = 0     

In [ ]:
#Matriz de confusión y accuracy
lista_valores_reales = conj_to_predict['active'].to_numpy()
lista_predicciones = conj_to_predict['prediccion'].to_numpy()

In [ ]:
matrix = confusion_matrix(lista_valores_reales, lista_predicciones)
matrix

In [ ]:
target_names = ['Prediccion -1', 'Prediccion 0', 'Prediccion 1']
print(classification_report(lista_valores_reales, lista_predicciones, target_names=target_names))